In [33]:
from ase.calculators.lj import LennardJones
from ase.calculators.plumed import Plumed
from ase.constraints import FixedPlane
from ase.md.langevin import Langevin
from ase.io import read
from ase import units

In [50]:
# Tutorial from https://gitlab.com/Sucerquia/ase-plumed_tutorial
timestep = 0.005
ps = 1000 * units.fs 

setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
         "c1: COORDINATIONNUMBER SPECIES=1-7 MOMENTS=2-3" +
         " SWITCH={RATIONAL R_0=1.5 NN=8 MM=16}",
         "PRINT ARG=c1.* STRIDE=10 FILE=COLVAR",
         "FLUSH STRIDE=100"]


In [54]:
from ase.calculators.lj import LennardJones
from ase.calculators.plumed import Plumed
from ase.constraints import FixedPlane
from ase.md.langevin import Langevin
from ase.io import read
from ase import units


timestep = 0.005

ps = 1000 * units.fs
setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
         "c1: COORDINATIONNUMBER SPECIES=1-7 MOMENTS=2-3" +
         " SWITCH={RATIONAL R_0=1.5 NN=8 MM=16}",
         "PRINT ARG=c1.* STRIDE=100 FILE=COLVAR",
         "FLUSH STRIDE=1000"]

atoms = read('isomerLJ.xyz')
# Constraint to keep the system in a plane
cons = [FixedPlane(i, [0, 0, 1]) for i in range(7)]
atoms.set_constraint(cons)
atoms.set_masses([1, 1, 1, 1, 1, 1, 1])

atoms.calc = Plumed(calc=LennardJones(rc=2.5, r0=3.),
                    input=setup,
                    timestep=timestep,
                    atoms=atoms,
                    kT=0.1)

dyn = Langevin(atoms, timestep, temperature_K=0.1/units.kB, friction=1,
               fixcm=False, trajectory='UnbiasMD.traj')

dyn.run(1000)

SyntaxError: keyword argument repeated: atoms (1785001900.py, line 27)

In [40]:

from ase.calculators.idealgas import IdealGas
from ase.calculators.plumed import Plumed
from ase.io import read, write
from ase import units


traj = read('UnbiasMD.traj', index=':')

atoms = traj[0]

write("final.xyz", atoms)

# timestep = 0.005
# ps = 1000 * units.fs
# setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
#          "c1: COORDINATIONNUMBER SPECIES=1-7 MOMENTS=2-3" +
#          " SWITCH={RATIONAL R_0=1.5 NN=8 MM=16}",
#          "PRINT ARG=c1.* STRIDE=100 FILE=COLVAR",
#          "FLUSH STRIDE=1000"]

# calc = Plumed(calc=IdealGas(),
#               input=setup,
#               timestep=timestep,
#               atoms=atoms,
#               kT=0.1)

# calc.write_plumed_files(traj)



PLUMED:                                               Cycles        Total      Average      Minumum      Maximum
PLUMED:                                                    1     0.001814     0.001814     0.001814     0.001814
PLUMED: 1 Prepare dependencies                           101     0.000063     0.000001     0.000000     0.000005
PLUMED: 2 Sharing data                                     2     0.000021     0.000011     0.000007     0.000014
PLUMED: 3 Waiting for data                                 2     0.000001     0.000001     0.000001     0.000001
PLUMED: 4 Calculating (forward loop)                       2     0.000093     0.000046     0.000035     0.000057
PLUMED: 5 Applying (backward loop)                         2     0.000005     0.000003     0.000001     0.000004
PLUMED: 6 Update                                           2     0.000076     0.000038     0.000010     0.000066


In [46]:
import numpy as np 

def coordination_numbers(atoms, d=1.5):
    cn = np.zeros(len(atoms))
    for i,atom_i in enumerate(atoms):
        cn[i] = 0
        for atom_j in atoms:
            if atom_i.index==atom_j.index:
                continue
            r_ij = atoms.get_distance(atom_i.index, atom_j.index, mic=False)
            cn[i] += (1-(r_ij/d)**8)/(1-(r_ij/d)**16)
    return cn

def moment(atoms, order, coordination_numbers):
    n_atoms = len(atoms)
    coord_num_exp = np.mean(coordination_numbers)
    moment = 0.0

    for i in range(n_atoms):
        moment += (coordination_numbers[i] - coord_num_exp)**order
    return moment/n_atoms

coord_num_val = coordination_numbers(atoms)
scm = moment(atoms, 2, coord_num_val)
tcm = moment(atoms, 3, coord_num_val)
print(f"{scm=}")
print(f"{tcm=}")

scm=np.float64(0.7579543969314642)
tcm=np.float64(1.335796192475372)
